# Model's architecture

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class ehrModel(nn.Module):
    
    def __init__(self, vocab_size, emb_dim, kernel_size):
        super(ehrModel, self).__init__()
        
        self.vocab_size = vocab_size
        self.emb_dim = emb_dim 
        self.kernel_size = kernel_size
        
        self.padding = int((kernel_size - 1) / 2) #input sequence length = output sequence length (if stride=dilation=1)
        
        self.embedding = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.cnn = nn.Conv2d(1, 1, kernel_size=(kernel_size, emb_dim), padding=(self.padding, 0))
        self.cnn2 = nn.Conv1d(1, 1, kernel_size=(kernel_size, 1), padding=(self.padding, 0))
        #self.FC = nn.Linear(1, 1)
    
    def forward(self, x):
        print("Shape original mat: {0}".format(x.shape))
        
        embeds = self.embedding(x)
        print("Size embedding matrix: {0}".format(embeds.shape))
        
        original_mat = embeds
        embeds = embeds.view(-1, 1, original_mat.shape[1], self.emb_dim) #reshape the temporal matrix with n_channels=emb_dim
        print("Embedding reshaping dimension: {0}".format(embeds.shape))
        
        out = F.relu(self.cnn(embeds))
        print("Output cnn: {0}".format(out.shape))
        out = F.max_pool2d(out, kernel_size=(kernel_size, out.shape[3]), padding=(self.padding, 0))
        print("Dimension after first maxpooling: {0}".format(out.shape))
        #out = out.view(-1, 1, out.shape[2])
        out = F.relu(self.cnn2(out))
        out = F.max_pool2d(out, kernel_size=(kernel_size, out.shape[3]), padding=(self.padding, 0))
        print("After Convolution dimension: {0}".format(out.shape))
        
        out = out.view(-1, out.shape[2]) #[N, [emb_dim * temporal_dim]] N is the batch_size
        print("Reshaping dimension: {0}".format(out.shape))
        
        #out = F.relu(self.FC(out))
        print("Output from CNN: {0}".format(out.shape))
        
        #out = out.view(-1,out.shape[1] * self.emb_dim) #[N, [emb_dim * temporal_dim]] N is the batch_size
        print("Reshaping dimension: {0}".format(out.shape))
        
        f_in = out.shape[1]
        print("AE input vector: {0}".format(f_in))
        f_out = x.shape[1]
        print(f_out)
        ehrAE = ae(f_in, f_out)
        #ehrAE = torch.nn.DataParallel(ehrAE, device_ids=[0, 2, 3])
        
        #ehrAE.cuda()
        #out = out.cuda()
        ehrAE.to(torch.device('cuda'))
        out = out.to(torch.device('cuda'))
        
        encoded_vect, out = ehrAE(out)
        #out = out.view(-1, 1, x.shape[1])
        #print(ehrAE.cuda())
        print("AE output dimension: {0}".format(out.shape))
        print("Dimension of the encoded vector: {0}".format(encoded_vect.shape))
        
        return(original_mat, out, encoded_vect)

In [3]:
class ae(nn.Module):
    def __init__(self, dim_in, dim_out):
        super(ae, self).__init__()
        self.dim_in = dim_in #input dimension (vector length)
        self.dim_out = dim_out
        
        h_dim = int(dim_in / 2)
        h2_dim = int(h_dim / 2)
        
        self.encoder = nn.Sequential(
        nn.Linear(dim_in, h_dim),
        nn.ReLU(True),
        nn.Linear(h_dim, h2_dim),
        nn.ReLU(True))
        self.decoder = nn.Sequential(
        nn.Linear(h2_dim, h_dim),
        nn.ReLU(True),
        nn.Linear(h_dim, dim_in),
        nn.ReLU(True),
        nn.Linear(dim_in, dim_out),
        nn.ReLU(True))
        
    def forward(self, x):
        x = self.encoder(x)
        encoded_vect = x
        x = self.decoder(x)
        #x.view(-1, 65979, 100)
        return(encoded_vect, x)